In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
DATA_FOLDER = 'data/'
assay = pd.read_csv(f'{DATA_FOLDER}BS_assay.csv')
# block_model = pd.read_csv(f'{DATA_FOLDER}BS_block_model.csv')
collar = pd.read_csv(f'{DATA_FOLDER}BS_collar.csv')
# old_lith = pd.read_csv(f'{DATA_FOLDER}BS_old_lith.csv')
# relog_geo = pd.read_csv(f'{DATA_FOLDER}BS_relog_geology.csv')
survey = pd.read_csv(f'{DATA_FOLDER}BS_survey.csv')

In [4]:
survey_dhs = survey.HOLEID.unique()
collar_dhs = collar.HOLEID.unique()
assay_dhs = assay.HOLEID.unique()

blacklisted_data = []
for dh in survey_dhs:
    if dh not in collar_dhs:
        blacklisted_data.append(dh)
        print(dh)
        
for dh in assay_dhs:
    if dh not in collar_dhs:
        blacklisted_data.append(dh)
        print(dh)

We will remove the drillholes above from the samples, as they have no collar data. We will only seem assay/DHs with collar, assay, and survey data.

In [6]:
blacklisted_data

['UG17-001',
 'UG17-002',
 'UG17-003',
 'UG17-006',
 'UG17-007',
 'UG17-008',
 'UG17-009',
 'UG17-004',
 'UG17-005',
 'UG17-010',
 'UG17-011',
 'UG17-012',
 'UG17-013',
 'UG17-014',
 'UG17-015',
 'UG17-016',
 'UG17-017']

In [37]:
collar.rename(columns = {
    'HOLEID': 'hole_id',
    'EAST': 'x', 
    'NORTH': 'y',
    'RL': 'z',
    'Azimuth': 'azi',
    'Dip': 'dip'
}, inplace=True)

survey.rename(columns = {
    'HOLEID': 'hole_id',
    'DEPTH': 'depth',
    'DIP': 'dip',
    'AZIMUTH': 'azi'
}, inplace=True)

columns_of_interest = ['x', 'y', 'z', 'azi', 'dip']

drillholes = {}
for i, row in collar.iterrows():
    hole_id = row.hole_id
    if hole_id not in blacklisted_data:
        dh = {
            'collar': row[columns_of_interest].to_dict(),
            'survey': survey.query("hole_id == @hole_id")  # TODO: should maybe clean this first?
        }
        drillholes[hole_id] = dh

    

In [38]:
drillholes.keys()

dict_keys(['1075s', '1077s', '1078s', '1079s', '1080s', '1198', '1199', '1200', '1201', '1202', '1203', '1204', '1208', '1209', '1210', '1211', '1212', '1213', '1214', '1215', '1216', '1217', '1218', '1219', '1220', '1221', '1222', '1223', '1224', '1225', '1226', '1227', '1228', '1229', '1230', '1231', '1232', '1233', '1234', '1235', '1236', '1237', '1238', '1239', '1240', '1241', '1242', '1243', '1244', '1245', '1246', '200601', '200602', '200603', '200604', '944', '945', '946', '947', '948', '949', '954', '955', '956', '957', '958', '959', '960', '961', '962', '963', '964', '984', '985', '986', '987', '990', '993', '994', 'BS0701', 'BS0702', 'BS0703', 'BS0704', 'BS0705', 'BS0706', 'BS0708', 'BS0709', 'BS0710', 'BS0711', 'BS0712', 'CS86-04', 'CS86-06', 'CS87-23', 'CS87-25', 'CS87-42', 'CS87-69', 'CS88-101', 'CS88-106', 'CS88-97', 'CS88-99', 'CS93-03', 'CS93-04', 'CS93-05', 'CS93-06', 'CS94-125', 'CS94-126', 'CS94-127', 'CS94-129', 'CS94-130', 'CS94-19', 'CS94-20', 'CS95-22', 'CS95-23'

In [51]:
for dh_id, dh in drillholes.items():
    dh_collar = dh['collar']
    x0, y0, z0 = dh_collar['x'], dh_collar['y'], dh_collar['z']
    
    dh_survey = dh['survey']
    # dh_survey['dist'] = 
    dh_survey['theta'] = np.add(90, np.multiply(-1, dh_survey.loc[:, 'azi'])) % 360
    dh_survey['dz'] = dh_survey['dist'] * np.sin(np.deg2rad(dh_survey.loc[:, 'dip']))


/var/folders/4h/m6_qv78d1ll6g0_5dn81rvh80000gp/T/ipykernel_62491/1410408512.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dh_survey['theta'] = np.add(90, np.multiply(-1, dh_survey.loc[:, 'azi'])) % 360
/var/folders/4h/m6_qv78d1ll6g0_5dn81rvh80000gp/T/ipykernel_62491/1410408512.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dh_survey['dz'] = dh_survey['dist'] * np.sin(np.deg2rad(dh_survey.loc[:, 'dip']))


In [52]:
drillholes['SK10-11']['collar']

{'x': 373665.19, 'y': 6280872.17, 'z': 575.48, 'azi': 45.0, 'dip': 210.0}

In [53]:
drillholes['SK10-11']['survey']

,hole_id,dist,dip,azi,theta,dz
449,SK10-11,0.00,45.0,210.0,240.0,0.000000
450,SK10-11,12.00,46.4,211.8,238.2,8.690062
451,SK10-11,60.00,46.8,218.2,231.8,43.738118
452,SK10-11,110.87,46.7,220.4,229.6,80.688166
